In [17]:
# 能處理載入、整理與視覺化等常見的資料應用套件
import pandas as pd

# 資料視覺化
import matplotlib.pyplot as plt
%matplotlib inline

# 分群 K-means
from sklearn.cluster import KMeans

# 以 matplotlib 為基礎建構的高階繪圖套件，讓使用者更加輕鬆地建立圖表，可視為是 matplotlib 的補強。
import seaborn as sns

from sklearn.metrics import silhouette_score

import numpy as np

#計算欄位內元素的頻度
from collections import Counter

import os

In [18]:
os.getcwd()

'E:\\III\\III\\Workplace\\TopicProject\\DataProcessing'

#### 欄位名稱
品牌：Brand；
食物名：Name；
品牌+食物名：BraName；
食用量：Intake； 
熱量：Calories, Cal；
蛋白質：Protein； 
脂肪：Fat；
飽和脂肪：Saturated fatty acid, SF；
不飽和脂肪 as Unsaturated fat, USF；
多（元）飽和脂肪：Polyunsaturated fat, PUSF；
單（元）飽和脂肪：Monounsaturated fat, MUSF；
膽固醇：Total cholesterol, TC；
碳水化合物：Carbohydrate, CHO；
糖：Sugar； 
纖維：Fiber；
鈉：Na；
鉀：K；
資訊來源：Resource； 
更新時間：UpdateTime； 
反鏈脂肪：Trans fat,TF。

In [19]:
# 讀入 csv 文字檔
FDADatabase_drink_csv = "../Data/Data_FDADatabase_drink_foodlabel.csv"
FDADatabase_drink = pd.read_csv(FDADatabase_drink_csv, encoding='utf-8')

food_csv = "../Data/Data_allFood_g_innerJoin2_foodlabel.csv"
food = pd.read_csv(food_csv, encoding='utf-8')

fatsecret_drink_csv = "../Data/Data_fatsecret_gNutrient_drink_ml_rmDupli_final_foodLabel.csv"
fatsecret_drink = pd.read_csv(fatsecret_drink_csv, encoding='utf-8')

myfitnesspal_drink_csv = "../Data/Data_myfitnesspal_drink_tai_foodlabel.csv"
myfitnesspal_drink = pd.read_csv(myfitnesspal_drink_csv, encoding='utf-8')


### 覽閱一下資料

In [20]:
#檢視一下資料
FDADatabase_drink.tail(2)

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,...,Ca_g,Mg_g,Fe_g,Zn_g,P_g,Cu_g,Mn_g,Resource,UpdateTime,FoodLabel
141,NaN,奶茶沖泡包(三合一),_奶茶沖泡包(三合一),100,g,451.9328,1.724,12.8500,12.4715,82.3467,...,0.01538,0.020284,0.00059,0.00038,0.25687,NaN,NaN,衛生福利部食品藥物管理署,20181207,9
142,NaN,部份脫脂調味奶粉(巧克力),NaN,100,g,390.6602,14.054,6.8704,4.8780,71.9373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,衛生福利部食品藥物管理署,20181207,9


In [21]:
#檢視一下資料
food.tail(2)

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,Sugar_g,Na_g,Resource,UpdateTime,FoodLabel
9162,星巴克,金幣巧克力零錢包,星巴克_金幣巧克力零錢包,42.0,g,217.0,2.8,11.6,6.9,25.7,0.0,0.0483,https://www.starbucks.com.tw/home/index.jspx,20190923,4
9163,星巴克,巧克力捲心棒,星巴克_巧克力捲心棒,36.0,g,198.0,2.4,12.3,11.7,19.8,0.0,0.0210,https://www.starbucks.com.tw/home/index.jspx,20190923,4


In [22]:
#檢視一下資料
fatsecret_drink.tail(2)

,Brand,Name,BraName,Intake_g,Intake_unit,Energy_kJ,Cal_kcal,Protein_g,Fat_g,SF_g,...,TC_g,CHO_g,Sugar_g,Fiber_g,Na_g,K_g,Resource,UpdateTime,FoodLabel,Unnamed: 22
1297,NaN,香蕉奶,_香蕉奶,100.0,g,395.0,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,73.0,0.08,NaN,Under Armour,2019,9,NaN
1298,天友,香蕉牛奶,天友_香蕉牛奶,100.0,g,327.0,78.0,2.4,2.9,NaN,...,NaN,10.5,NaN,NaN,NaN,NaN,fatsecret中國,2015052,9,NaN


In [23]:
#檢視一下資料
myfitnesspal_drink.tail(2)

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,CHO_g,Fiber_g,Sugar_g,Fat_g,...,PUSF_g,MUSF_g,TF_g,Protein_g,Na_g,K_g,TC_g,Resource,UpdateTime,FoodLabel
712,NaN,飲用水,_飲用水,100.0,g,349,59,10,9,6,...,0,0,0,11,0.06,0.0,0.0,Under Armour,2019,9
713,NaN,白開水,_白開水,100.0,g,29,7,0,0,0,...,0,0,0,0,0.00,0.0,0.0,Under Armour,2019,9


### 合併資料
```python
new_dataframe = pd.concat([df1,df2], join='outer', ignore_index=True)
```
join = 
<br>outer (default): 結合所有的欄位，沒有資料已Nan表示，各dataframe欄位取聯集的概念。
<br>inner: 會直接把沒有完整資料的刪除掉，各dataframe欄位取交集的概念。
<br>
<br>ignore_index = 
<br>True: 重整左側的index。
<br>False: 不重整index，保留原資料之index。

In [24]:
# fatsecret,mos, starbucks, subway 欄位取聯集
dfo = pd.concat([FDADatabase_drink, food, fatsecret_drink, myfitnesspal_drink], join='outer', ignore_index=True) #df_outer as dfo
dfo.tail()

c:\users\big data\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,BraName,Brand,CHO_g,Ca_g,Cal_kcal,Cu_g,Energy_kJ,Fat_g,Fe_g,Fiber_g,...,P_g,Protein_g,Resource,SF_g,Sugar_g,TC_g,TF_g,Unnamed: 22,UpdateTime,Zn_g
11315,_溫水,NaN,17.0,NaN,76,NaN,NaN,0.0,NaN,0.0,...,NaN,2.0,Under Armour,0.0,0.0,0.0,0.0,NaN,2019,NaN
11316,_鹽水,NaN,14.0,NaN,623,NaN,NaN,55.0,NaN,0.0,...,NaN,16.0,Under Armour,0.0,0.0,0.0,0.0,NaN,2019,NaN
11317,_純淨水,NaN,15.0,NaN,70,NaN,NaN,0.0,NaN,0.0,...,NaN,3.0,Under Armour,0.0,0.0,0.0,0.0,NaN,2019,NaN
11318,_飲用水,NaN,59.0,NaN,349,NaN,NaN,6.0,NaN,10.0,...,NaN,11.0,Under Armour,0.0,9.0,0.0,0.0,NaN,2019,NaN
11319,_白開水,NaN,7.0,NaN,29,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,Under Armour,0.0,0.0,0.0,0.0,NaN,2019,NaN


In [25]:
# fatsecret,mos, starbucks, subway 欄位取交集
# 因為mos, Subway沒有intake，無法單位化，所以就不合併了。
dfi= pd.concat([FDADatabase_drink, food, fatsecret_drink, myfitnesspal_drink], join='inner', ignore_index=True) #df_inner as dfi

#重整index
dfi = dfi.reset_index(drop=True)

dfi.tail()

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,Sugar_g,Na_g,Resource,UpdateTime,FoodLabel
11315,NaN,溫水,_溫水,100.0,g,76,2.0,0.0,0.0,17.0,0.0,0.002,Under Armour,2019,9
11316,NaN,鹽水,_鹽水,100.0,g,623,16.0,55.0,0.0,14.0,0.0,0.000,Under Armour,2019,9
11317,NaN,純淨水,_純淨水,100.0,g,70,3.0,0.0,0.0,15.0,0.0,0.000,Under Armour,2019,9
11318,NaN,飲用水,_飲用水,100.0,g,349,11.0,6.0,0.0,59.0,9.0,0.060,Under Armour,2019,9
11319,NaN,白開水,_白開水,100.0,g,29,0.0,0.0,0.0,7.0,0.0,0.000,Under Armour,2019,9


In [26]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11320 entries, 0 to 11319
Data columns (total 15 columns):
Brand          4097 non-null object
Name           11320 non-null object
BraName        11319 non-null object
Intake_g       11320 non-null float64
Intake_unit    11320 non-null object
Cal_kcal       11319 non-null object
Protein_g      11311 non-null float64
Fat_g          11298 non-null float64
SF_g           8049 non-null float64
CHO_g          11318 non-null float64
Sugar_g        7136 non-null float64
Na_g           10768 non-null float64
Resource       11320 non-null object
UpdateTime     11320 non-null int64
FoodLabel      11320 non-null int64
dtypes: float64(7), int64(2), object(6)
memory usage: 1.3+ MB


#### ==============================   以下先不做 ============================== 

#### 單位化
營養成分/食用量
```python
df[i]  = df.apply(lambda x: x [i]  / x['Intake_g'], axis=1)
```

#### 建立新的df，以做運算
dfa = dfi[['Intake_g', 'Cal_kcal', 'Protein_g', 'Fat_g', 'SF_g', 'CHO_g', 'Sugar_g','Na_g']]

list_ = ['Intake_g', 'Cal_kcal', 'Protein_g', 'Fat_g', 'SF_g', 'CHO_g', 'Sugar_g','Na_g']
for i in list_:
    dfi[i]  = dfa.apply(lambda x: x [i]  / x['Intake_g'], axis=1)

dfi.head()

#### ==============================  以上先不做 ============================== 

#### 檢查檔案是否合理

dfi['intake']須大於dfi['Protein_g']+dfi['Fat_g']+dfi['CHO_g']

In [27]:
dfi = dfi[dfi['Intake_g'] > (dfi['Protein_g'] + dfi['Fat_g'] + dfi['CHO_g'])]

In [28]:
dfi.tail()

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,Sugar_g,Na_g,Resource,UpdateTime,FoodLabel
11315,NaN,溫水,_溫水,100.0,g,76,2.0,0.0,0.0,17.0,0.0,0.002,Under Armour,2019,9
11316,NaN,鹽水,_鹽水,100.0,g,623,16.0,55.0,0.0,14.0,0.0,0.000,Under Armour,2019,9
11317,NaN,純淨水,_純淨水,100.0,g,70,3.0,0.0,0.0,15.0,0.0,0.000,Under Armour,2019,9
11318,NaN,飲用水,_飲用水,100.0,g,349,11.0,6.0,0.0,59.0,9.0,0.060,Under Armour,2019,9
11319,NaN,白開水,_白開水,100.0,g,29,0.0,0.0,0.0,7.0,0.0,0.000,Under Armour,2019,9


#### 刪除df_drink['BraName']重複值
```python
df = df.drop_duplicates(subset='BraName', keep='first') 
```
dataframe.drop_duplicates(subset='column', keep='first', inplace = False) 
<br>subset：以哪個欄位當主；keep：保留哪一值，{'first', 'last', 'False'（刪除所有重複）}, default 'first'；
<br>inplace : boolean, default False. Whether to drop duplicates in place or to return a copy

In [29]:
dfi = dfi.drop_duplicates(subset='BraName', keep='last') #取最後的，因為以官網為主，資料庫的資料為輔

#重整index
dfi = dfi.reset_index(drop=True)

dfi

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,Sugar_g,Na_g,Resource,UpdateTime,FoodLabel
0,NaN,"烏龍茶(去冰,微糖)","_烏龍茶(去冰,微糖)",100.0,g,15.5005,0.0708,0.0121,NaN,3.7771,3.5579,NaN,衛生福利部食品藥物管理署,20181207,9
1,NaN,"烏龍茶(去冰,半糖)","_烏龍茶(去冰,半糖)",100.0,g,21.0873,0.0633,0.0029,NaN,5.2020,4.9109,NaN,衛生福利部食品藥物管理署,20181207,9
2,NaN,"烏龍茶(去冰,全糖)","_烏龍茶(去冰,全糖)",100.0,g,35.6678,0.0633,0.0114,NaN,8.8280,8.1667,NaN,衛生福利部食品藥物管理署,20181207,9
3,NaN,鮮奶茶(無糖),_鮮奶茶(無糖),100.0,g,26.5077,2.0180,1.4317,0.9615,1.3876,1.8110,0.011872,衛生福利部食品藥物管理署,20181207,9
4,NaN,奶茶(三合一),_奶茶(三合一),100.0,g,41.97,0.4580,0.2500,0.1945,9.4720,NaN,0.018040,衛生福利部食品藥物管理署,20181207,9
5,NaN,"烏龍奶茶(去冰,微糖)","_烏龍奶茶(去冰,微糖)",100.0,g,59.3231,0.2473,2.8983,NaN,8.0623,3.7811,NaN,衛生福利部食品藥物管理署,20181207,9
6,NaN,"烏龍奶茶(去冰,半糖)","_烏龍奶茶(去冰,半糖)",100.0,g,62.3074,0.2410,2.5606,NaN,9.5745,4.7131,NaN,衛生福利部食品藥物管理署,20181207,9
7,NaN,"烏龍奶茶(去冰,全糖)","_烏龍奶茶(去冰,全糖)",100.0,g,74.7962,0.2348,2.7550,NaN,12.2655,8.5539,NaN,衛生福利部食品藥物管理署,20181207,9
8,NaN,多多綠茶(全糖),_多多綠茶(全糖),100.0,g,49.019,1.0590,0.0882,NaN,10.9973,10.2070,0.010081,衛生福利部食品藥物管理署,20181207,9
9,NaN,龜苓茶,_龜苓茶,100.0,g,33.0144,0.0000,0.0000,NaN,8.2536,NaN,0.017840,衛生福利部食品藥物管理署,20181207,9


In [30]:
dfi.tail()

,Brand,Name,BraName,Intake_g,Intake_unit,Cal_kcal,Protein_g,Fat_g,SF_g,CHO_g,Sugar_g,Na_g,Resource,UpdateTime,FoodLabel
10984,NaN,溫水,_溫水,100.0,g,76,2.0,0.0,0.0,17.0,0.0,0.002,Under Armour,2019,9
10985,NaN,鹽水,_鹽水,100.0,g,623,16.0,55.0,0.0,14.0,0.0,0.000,Under Armour,2019,9
10986,NaN,純淨水,_純淨水,100.0,g,70,3.0,0.0,0.0,15.0,0.0,0.000,Under Armour,2019,9
10987,NaN,飲用水,_飲用水,100.0,g,349,11.0,6.0,0.0,59.0,9.0,0.060,Under Armour,2019,9
10988,NaN,白開水,_白開水,100.0,g,29,0.0,0.0,0.0,7.0,0.0,0.000,Under Armour,2019,9


In [31]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10989 entries, 0 to 10988
Data columns (total 15 columns):
Brand          4078 non-null object
Name           10989 non-null object
BraName        10988 non-null object
Intake_g       10989 non-null float64
Intake_unit    10989 non-null object
Cal_kcal       10989 non-null object
Protein_g      10989 non-null float64
Fat_g          10989 non-null float64
SF_g           7754 non-null float64
CHO_g          10989 non-null float64
Sugar_g        6850 non-null float64
Na_g           10447 non-null float64
Resource       10989 non-null object
UpdateTime     10989 non-null int64
FoodLabel      10989 non-null int64
dtypes: float64(7), int64(2), object(6)
memory usage: 1.3+ MB


缺值補0
```python
df= df.fillna(0)
```

In [32]:
fillna_list = ['Intake_g', 'Cal_kcal', 'Protein_g', 'Fat_g', 'SF_g', 'CHO_g', 'Sugar_g','Na_g']
dfi[fillna_list] = dfi[fillna_list].fillna(0) 

In [34]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10989 entries, 0 to 10988
Data columns (total 15 columns):
Brand          4078 non-null object
Name           10989 non-null object
BraName        10988 non-null object
Intake_g       10989 non-null float64
Intake_unit    10989 non-null object
Cal_kcal       10989 non-null object
Protein_g      10989 non-null float64
Fat_g          10989 non-null float64
SF_g           10989 non-null float64
CHO_g          10989 non-null float64
Sugar_g        10989 non-null float64
Na_g           10989 non-null float64
Resource       10989 non-null object
UpdateTime     10989 non-null int64
FoodLabel      10989 non-null int64
dtypes: float64(7), int64(2), object(6)
memory usage: 1.3+ MB


In [36]:
# 存一下檔
dfi.to_csv('../Data/Data_allFood&Drink_innerJoin_foodLabel.csv', encoding='utf-8', index=False)